[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kwaulere/Sber_DS/blob/master/Home_work/Home_work_4.ipynb)

In [8]:
from pandas.core.common import SettingWithCopyWarning
import warnings
import pandas as pd
import random
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [17]:
df=pd.read_csv("/Users/crap/Sber/Data/Химический анализ родника в Нескучном саду.csv", sep=';')

In [18]:
df.head(5)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"


##### Разделим выборку по виду показателя 'в пределах' и 'не более'

In [19]:
dataFrameOut = df[df['Норматив'].str.contains('в пределах')]
dataFrameOut['borders'] = dataFrameOut['Норматив'].str.findall(r'\d{1,2}')
dataFrameOut['min'] = dataFrameOut['borders'].apply(lambda x: x[0])
dataFrameOut['max'] = dataFrameOut['borders'].apply(lambda x: x[1])
dataFrameOut[['min','max','Результат анализа']] = dataFrameOut[['min','max','Результат анализа']].astype(float)
dataFrameOut.loc[(dataFrameOut['max'] > dataFrameOut['Результат анализа']) & (dataFrameOut['min'] < dataFrameOut['Результат анализа']), 'Выводы'] = 'Результат в норме'
dataFrameOut.loc[(dataFrameOut['max'] < dataFrameOut['Результат анализа']) | (dataFrameOut['min'] > dataFrameOut['Результат анализа']), 'Выводы'] = 'Результат провален'
final = dataFrameOut[['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив', 'Выводы']]

In [20]:
dataFrameOut = df[df['Норматив'].str.contains('не более')]
dataFrameOut['границы'] = dataFrameOut['Норматив'].str.replace('не более ','').str.replace('-',',').str.replace(',','.')
dataFrameOut['границы'] = dataFrameOut['границы'].astype(float)
dataFrameOut['Результат анализа'] = dataFrameOut['Результат анализа'].str.replace('б/цвета','0')
dataFrameOut['Результат анализа'] = dataFrameOut['Результат анализа'].astype(float)
dataFrameOut.loc[dataFrameOut['границы'] >= dataFrameOut['Результат анализа'], 'Выводы'] = 'Результат в норме'
dataFrameOut.loc[dataFrameOut['границы'] < dataFrameOut['Результат анализа'], 'Выводы'] = 'Результат провален'
dataFrameOut = dataFrameOut[['Показатель', 'Единица измерений', 'Результат анализа', 'Норматив', 'Выводы']]
final = pd.concat([final, dataFrameOut])

####  В выводах получаем результат проверки показателей, если тест провален это будет отмечено

In [21]:
final

,Показатель,Единица измерений,Результат анализа,Норматив,Выводы
0,pH,единицы pH,8.400,в пределах 6-9,Результат в норме
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Результат в норме
1,Запах,баллы,1.000,не более 2-3,Результат в норме
2,Цветность,градусы,0.000,не более 30,Результат в норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",Результат в норме
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Результат в норме
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,Результат в норме
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",Результат в норме
8,Хлориды (Cl),мг/дм3,200.000,не более 350,Результат в норме
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,Результат в норме


Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?



In [131]:
n = 1000000
result = {'success':0, 'fail':0}
for i in range(n):
    box = ['apple']*4 + ["orange"]*5

    a = box.pop(random.randrange(0,len(box)))
    b = box.pop(random.randrange(0,len(box)))
    c = box.pop(random.randrange(0,len(box)))

    if a == b == c == 'orange':
        result['success'] += 1
    else:
        result['fail'] += 1
print(f'{result["success"]/result["fail"]}')

0.13528769325434759


Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [132]:
n = 1000000
result = {'success':0, 'fail':0}
for i in range(n):
    box = ['stndrt']*7 + ["nostndrt"]*3
    a = box.pop(random.randrange(0,len(box)))
    b = random.choice(box)

    if a == "nostndrt" and b == 'stndrt':
        result['success'] += 1
    else:
        result['fail'] += 1
print(f'{result["success"]/result["fail"]}')


0.3046263354481326
